## Загружаем необходимые библиотеки

In [ ]:
import os

import random
import pandas as pd
import numpy as np
import shutil
from tqdm import tqdm
import time

import matplotlib
import plotly
import PIL
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from google.colab.patches import cv2_imshow

from torch.utils.data import Dataset, DataLoader
import glob, matplotlib.pyplot as plt, matplotlib.patches as patches

import yaml
import torch
import json
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from sklearn.model_selection import train_test_split

## Загрузка датасета




In [ ]:
!git clone 'https://github.com/MelnikDM/SVO.git'

Cloning into 'SVO'...
remote: Enumerating objects: 3851, done.
remote: Counting objects: 100% (317/317), done.
remote: Compressing objects: 100% (289/289), done.
remote: Total 3851 (delta 26), reused 306 (delta 17), pack-reused 3534
Receiving objects: 100% (3851/3851), 547.95 MiB | 38.92 MiB/s, done.
Resolving deltas: 100% (1769/1769), done.
Updating files: 100% (3569/3569), done.


## Обработка аннотаций

И разбиение выборок на обучающую, валидационную и тестовую





In [ ]:
image_list = os.listdir('/content/SVO/LA_ObjectDetection/data/Images')
train_list, test_list = train_test_split(image_list, test_size=0.2)
val_list, test_list = train_test_split(test_list, test_size=0.5)

print('Обучающая выборка: ',len(train_list))
print('Проверочная выборка: ',len(val_list))
print('Тестовая выборка:' ,len(test_list))

Обучающая выборка:  1324
Проверочная выборка:  165
Тестовая выборка: 166


In [ ]:
!mkdir '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/'

In [ ]:
!mkdir '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/test_images/'
!mkdir '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/train_images/'
!mkdir '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/train_xmls/'
!mkdir '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/valid_images/'
!mkdir '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/valid_xmls/'

In [ ]:
!mkdir '/content/outputs/'

In [ ]:
def move_data(name_list, folder, newfolder, mode, img=True):
  """
  Функция перемещает файлы в указанную папку.
  name_list - список с названиями файлов для перемещения
  folder    - путь к папке откуда берем файл
  mode      - название папки куда перемещаем
  img       - указывает какой тип файла мы перемещаем
  """
  format = 'jpg' if img==True else 'xml'
  for name in name_list:
    name = name[:-3]+format
    shutil.move(folder + name, newfolder + '{}/'.format(mode) + name)

In [ ]:
move_data(train_list, '/content/SVO/LA_ObjectDetection/data/Images/', '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/', 'train_images', img=True)
move_data(val_list, '/content/SVO/LA_ObjectDetection/data/Images/', '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/', 'valid_images', img=True)

In [ ]:
move_data(train_list, '/content/SVO/LA_ObjectDetection/data/Annotations/', '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/', 'train_xmls', img=False)
move_data(val_list, '/content/SVO/LA_ObjectDetection/data/Annotations/', '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/', 'valid_xmls', img=False)

In [ ]:
move_data(test_list, '/content/SVO/LA_ObjectDetection/data/Images/', '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/', 'test_images', img=True)

## Обучение модели

In [ ]:
%cd /content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP

/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP


In [ ]:
!python train.py --model fasterrcnn_resnet50_fpn --epochs 25 --batch-size 4 --workers 4 --project-name LA_detection_fasterrcnn_resnet50_fpn_v2 --config /content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data_configs/LA.yaml --use-train-aug

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Number of training samples: 1324
Number of validation samples: 165

Building model from scratch...
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100% 160M/160M [00:01<00:00, 112MB/s]
FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWit

In [ ]:
!mkdir '/content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/data/inference_data/'

In [ ]:
!zip -r /content/FRCNN.zip /content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/content/outputs/training/smoke_detection_fasterrcnn_resnet50_fpn_v2

  adding: content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/content/outputs/training/smoke_detection_fasterrcnn_resnet50_fpn_v2/ (stored 0%)
  adding: content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/content/outputs/training/smoke_detection_fasterrcnn_resnet50_fpn_v2/loss_obj.png (deflated 16%)
  adding: content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/content/outputs/training/smoke_detection_fasterrcnn_resnet50_fpn_v2/train_loss_iter.png (deflated 11%)
  adding: content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/content/outputs/training/smoke_detection_fasterrcnn_resnet50_fpn_v2/train_loss_epoch.png (deflated 16%)
  adding: content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/content/outputs/training/smoke_detection_fasterrcnn_resnet50_fpn_v2/last_model_state.pth (deflated 7%)
  adding: content/SVO/LA_ObjectDetection/Faster_RCNN_ResNet50_maP/content/outputs/training/smoke_detection_fasterrcnn_resnet50_fpn_v2/image_3_1.jpg (deflated 5%)
  adding: content/SVO

In [ ]:
from google.colab import files
files.download("/content/FRCNN.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>